In [13]:
import json
import requests

query = "best res"

cursor_pos = len(query)
url_friendly_query = query.replace(" ", "%20")

url = f"https://www.google.com/complete/search?q={url_friendly_query}%20reddit&cp={cursor_pos}&client=gws-wiz&xssi=t&hl=en&authuser=0&dpr=2"

res = requests.get(url).text[5:]
parsed = json.loads(res)[0]

In [14]:
suggestions = [element[0].replace("reddit", "").replace(" </b>", "</b>") for element in parsed]

In [15]:
suggestions

['best res<b>ume builder</b>',
 'best res<b>ident evil game</b>',
 'best res<b>ume template</b>',
 'best res<b>istance bands</b>',
 'best res<b>taurants nyc</b>',
 'best res<b>ume writing service</b>',
 'best res<b>taurants chicago</b>',
 'best res<b>taurants montreal</b>',
 'best res<b>p canada</b>',
 'best res<b>taurants in rome</b>']

In [56]:
from googlesearch import search
submission_ids = []
for res in search("best git mergetool reddit", safe=None):
    if "/comments/" in res:
        submission_id = res.split("/")[6]
        submission_ids.append(submission_id)

In [57]:
import praw

r = praw.Reddit(
    client_id="2gs3ULbVK-pGh7INla0gWg",
    client_secret="g6S2REUlWw8fBBIgJiaFrqIkZUVICQ",
    user_agent="web:tech.redditrecs:v1.0.0 (by u/matetoes)",
)

In [58]:
submission_ids

['158zdir',
 'icptpk',
 '6hjm92',
 'dvs1jt',
 '13si22h',
 '6hjm92',
 'dvs1jt',
 'b0jjgw',
 'dvs1jt',
 'u82rmz',
 '13si22h',
 '18hw0zp']

In [61]:
from multiprocess import Pool
import time

submissions = []

def jsonify(comment):
    return {
        "body": comment.body,
        "score": comment.score,
        "author": str(comment.author),
        "replies": [jsonify(reply) for reply in comment.replies],
    }

def get(submission_id):
    # let's time how long this takes
    start = time.time()
    comments = []
    submission = r.submission(submission_id)
    submission.comments.replace_more(limit=None)
    # get title and body of submission
    title = submission.title
    selftext = submission.selftext
    for comment in submission.comments:
        comments.append(jsonify(comment))
    result = {
        "title": title,
        "selftext": selftext,
        "comments": comments,
    }
    end = time.time()
    print(f"Submission {submission_id} took {end - start} seconds")
    return result

with Pool(len(submission_ids)) as pool:
    submissions = pool.map(get, submission_ids)

Submission 6hjm92 took 0.5821151733398438 seconds
Submission dvs1jt took 0.6117157936096191 secondsSubmission 6hjm92 took 0.7126731872558594 seconds
Submission 158zdir took 0.7409169673919678 seconds

Submission b0jjgw took 0.7538909912109375 seconds
Submission icptpk took 0.7272889614105225 seconds
Submission u82rmz took 0.7678050994873047 seconds
Submission dvs1jt took 0.7784738540649414 seconds
Submission dvs1jt took 0.8248569965362549 seconds
Submission 18hw0zp took 1.6788458824157715 seconds
Submission 13si22h took 2.033061981201172 seconds
Submission 13si22h took 2.5215909481048584 seconds


In [69]:
import json

len(json.dumps(submissions[0]["comments"][0]))

# lets find the max
max_len = 0
max_comment = None
for submission in submissions:
    for comment in submission["comments"]:
        if len(json.dumps(comment)) > max_len:
            max_len = len(json.dumps(comment))
            max_comment = comment

max_len

27923